<a href="https://www.kaggle.com/code/dsptlp/canada-age-pyramid-bargraph?scriptVersionId=169441232" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Canada Age Pyramid

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import math
import matplotlib.ticker as mtick
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Rectangle

In [2]:
def download_csv(url, destination):
    response = requests.get(url)
    with open(destination, 'wb') as file:
        file.write(response.content)

csv_url = 'https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=9810002701&latestN=0&startDate=20210101&endDate=20210101&csvLocale=en&selectedMembers=%5B%5B1%5D%2C%5B%5D%2C%5B%5D%5D&checkedLevels=1D1%2C1D2%2C1D3%2C1D4%2C1D5%2C2D1%2C2D2'
destination_path = 'data_file.csv'

download_csv(csv_url, destination_path)

In [3]:
df = pd.read_csv(destination_path, encoding='utf-8')
df.head(5)

REF_DATE     GEO           DGUID  \
0      2021  Canada  2021A000011124   
1      2021  Canada  2021A000011124   
2      2021  Canada  2021A000011124   
3      2021  Canada  2021A000011124   
4      2021  Canada  2021A000011124   

  Age (in single years), average age and median age (128)      Gender (3)  \
0                                        Total - Age       Total - Gender   
1                                        Total - Age                 Men+   
2                                        Total - Age               Women+   
3                                      0 to 14 years       Total - Gender   
4                                      0 to 14 years                 Men+   

   UOM  UOM_ID SCALAR_FACTOR  SCALAR_ID  VECTOR COORDINATE       VALUE  \
0  NaN       0         units          0     NaN      1.1.1  36991980.0   
1  NaN       0         units          0     NaN      1.1.2  18226235.0   
2  NaN       0         units          0     NaN      1.1.3  18765745.0   
3  NaN       0         units          0     NaN      1.2.1   6012795.0   
4  NaN       0         units          0     NaN      1.2.2   3086510.0   

   STATUS  SYMBOL  TERMINATED  DECIMALS  
0     NaN     NaN         NaN         0  
1     NaN     NaN         NaN         0  
2     NaN     NaN         NaN         0  
3     NaN     NaN         NaN         0  
4     NaN     NaN         NaN         0

In [4]:
df = df[df['Age (in single years), average age and median age (128)'].str.isnumeric()]
df = df[df['Gender (3)'].isin(['Men+','Women+'])]
df = df.pivot(index='Age (in single years), average age and median age (128)', columns='Gender (3)', values='VALUE')
df = df.reset_index()

df['Age'] = df['Age (in single years), average age and median age (128)'].astype(int)

df['Men+'] = df['Men+'] / -1
df['Women+'] = df['Women+'] / 1

df = df.sort_values(by='Age', ascending=False)

df

Gender (3) Age (in single years), average age and median age (128)      Men+  \
98                                                         99        -1280.0   
97                                                         98        -1960.0   
96                                                         97        -2935.0   
95                                                         96        -4250.0   
94                                                         95        -7240.0   
..                                                        ...            ...   
44                                                          5      -205945.0   
33                                                          4      -200440.0   
22                                                          3      -193230.0   
11                                                          2      -186855.0   
0                                                           1      -181865.0   

Gender (3)    Women+  Age  
98            4665.0   99  
97            6590.0   98  
96            9185.0   97  
95           12075.0   96  
94           18205.0   95  
..               ...  ...  
44          196155.0    5  
33          189875.0    4  
22          183800.0    3  
11          178470.0    2  
0           173525.0    1  

[99 rows x 4 columns]

In [5]:
sns.set(style="whitegrid", font_scale=1)
sns.set_palette("pastel")  # Choose a light color palette

fig, ax = plt.subplots(1, 1, figsize = (8, 5), dpi=500, frameon =True)
ax.set_facecolor('white')  # Set a light background color

ax1 = sns.barplot(x='Men+', y='Age (in single years), average age and median age (128)', data=df, color="blue",)
ax2 = sns.barplot(x='Women+', y='Age (in single years), average age and median age (128)', data=df, color="red")

y_ticks = ax.get_yticks()[::2]
plt.yticks(y_ticks,rotation=0)

plt.xticks(rotation=90) 

ax.set_ylabel('Age', fontsize=20)    
ax.set_xlabel('Population (x 1,000)', fontsize=20)
ax.set_title('Canada - Age (in single years) and gender', fontsize=20)

ax.tick_params(axis='x', labelsize=5)
ax.tick_params(axis='y', labelsize=5)

ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:,.0f}'.format(abs(x))))

plt.xlim(-300000,300000)

legend_elements = [
    Line2D([0], [0], color='b', lw=5, label='Men'),
    Line2D([0], [0], color='r', lw=5, label='Women')
]

ax.legend(handles=legend_elements, loc='upper right', fontsize=8)

# Specify the age group you want to highlight (adjust as needed)
highlight_age_group = '25'  # Change this to the desired age group

# Find the index of the specified age group in the x-axis labels
highlight_column_index = df[df['Age (in single years), average age and median age (128)'] == highlight_age_group].index[0]

# Add a red box around the specified age group
highlighted_bar = ax.patches[highlight_column_index]
highlighted_box = Rectangle((highlighted_bar.get_x(), 0), width=highlighted_bar.get_width(), height=ax.get_ylim()[1], linewidth=2, edgecolor='red', facecolor='none')
ax.add_patch(highlighted_box)

footnote_text = "Source: Statistics Canada. Table 98-10-0027-01  Age (in single years), average age and median age and gender: Canada and forward sortation areas ©DOI: https://doi.org/10.25318/9810002701-eng"
plt.figtext(0.06, -0.06, footnote_text, ha="left", fontsize=5, bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.3'))    

fig.savefig('graph4.png',transparent=True,  bbox_inches='tight',pad_inches=0.15,dpi=400)